In [ ]:
import os
from dotenv import load_dotenv

# Load variables from .env into environment
load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL") or (
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}"
    f"@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

print(DATABASE_URL)  # sanity check

In [ ]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Build DB URL
DATABASE_URL = os.getenv("DATABASE_URL") or (
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}"
    f"@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

def test_connection():
    try:
        engine = create_engine(DATABASE_URL)
        with engine.connect() as conn:
            result = conn.execute(text("SELECT 1")).scalar()
            if result == 1:
                print("✔ Connection Successful! DB is reachable.")
    except Exception as e:
        print(f"❌ Connection Failed:\n{e}")

if __name__ == "__main__":
    test_connection()


In [ ]:
import pandas as pd

df = pd.read_excel(
    r"C:\Users\trilo\OneDrive\Desktop\Report_Transactions\3380.xlsx"
)
df["Period"] = pd.to_datetime(df["Period"]).dt.date

print("🔍 Sample rows from Excel:")
print(df.head(10))     # first 10 rows


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# ============================================================
# 🔐 Load Environment Variables
# ============================================================
load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL") or (
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@"
    f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

# ============================================================
# 📂 Excel File Path
# ============================================================
file_path = r"C:\Users\trilo\OneDrive\Desktop\Report_Transactions\3380.xlsx"

# ============================================================
# 🗄 Table Name (change if needed)
# ============================================================
TABLE_NAME = "finance_data"

# ============================================================
# 🚀 Push Excel → PostgreSQL
# ============================================================
def push_excel_to_db():
    # Read Excel
    df = pd.read_excel(file_path)

    # Optional: clean column names (recommended)
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace(r"[^\w_]", "", regex=True)
    )

    # Create DB engine
    engine = create_engine(DATABASE_URL)

    # Push to DB
    df.to_sql(
        TABLE_NAME,
        engine,
        if_exists="append",   # use "replace" if you want to overwrite
        index=False,
        method="multi",
        chunksize=5000
    )

    print(f"✅ {len(df)} rows pushed successfully to '{TABLE_NAME}'")

# ============================================================
# ▶ Run
# ============================================================
if __name__ == "__main__":
    push_excel_to_db()


In [ ]:
import subprocess

PG_BIN = r"C:\Program Files\PostgreSQL\16\bin"
CREATEDB = fr"{PG_BIN}\createdb.exe"
PG_RESTORE = fr"{PG_BIN}\pg_restore.exe"

BACKUP_FILE = r"C:\Users\trilo\OneDrive\Desktop\Report_Transactions\Backup\3380.dat"
DB_NAME = "report_mqba_restore"

subprocess.run([CREATEDB, DB_NAME], capture_output=True)
subprocess.run([PG_RESTORE, "-d", DB_NAME, BACKUP_FILE], check=True)

print("✅ Database restored")
